In [1]:
%pylab inline
import pandas as pd
import numpy as np
import sklearn
from matplotlib import pyplot as plt
import nltk
from nltk.collocations import *
import re

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset = pd.read_csv("Emails.csv")

In [3]:
texts = list(dataset["RawText"])

In [4]:
_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

for i, text in enumerate(texts):
    text = re.sub(r'[^\w]', ' ', text)
    words = text.split(" ")
    prepared_words = [word.lower() for word in words if len(word) > 1 and not contains_digits(word)]
    texts[i] = prepared_words

Я очищаю от всех не буквенных символов (знаки препинания, цифры и тд), а также отбрасываю слова короче 2 букв

In [5]:
#frequent bigrams
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(texts)
res = finder.score_ngrams(bigram_measures.raw_freq)
print res[:50]

[(('of', 'state'), 0.010254766106249407), (('department', 'of'), 0.009887575680712458), (('doc', 'no'), 0.009656125620611865), (('case', 'no'), 0.009653578214290004), (('state', 'case'), 0.009650302977590466), (('no', 'doc'), 0.00964848340164628), (('no', 'date'), 0.009647755571268605), (('unclassified', 'department'), 0.00964702774089093), (('of', 'the'), 0.005122470198075398), (('state', 'gov'), 0.004023082412597579), (('in', 'the'), 0.003752693427291383), (('release', 'in'), 0.002880752634836946), (('date', 'release'), 0.0027209938669373224), (('message', 'from'), 0.0026529417266247268), (('original', 'message'), 0.002632198560860994), (('subject', 're'), 0.002450604881631126), (('to', 'the'), 0.0023800053349966686), (('pm', 'to'), 0.0020430198701332255), (('date', 'unclassified'), 0.0019695090019880685), (('on', 'the'), 0.0018970898794094238), (('gov', 'sent'), 0.00178937098351355), (('clintonemail', 'com'), 0.0017784535278484279), (('in', 'part'), 0.00161651126881578), (('to', 'su

In [6]:
#max pmi bigrams
res = finder.score_ngrams(bigram_measures.pmi)
print res[:50]

[(('_ali', 'taajw'), 21.389894397579603), (('_apecrets', 'prg'), 21.389894397579603), (('_lewda', 'lysched'), 21.389894397579603), (('_mrsiata', 'nizenmenvarmars'), 21.389894397579603), (('_reunite', 'arriericll'), 21.389894397579603), (('_rmuacartatmott', 'rmaamaxatiamasm'), 21.389894397579603), (('_s_', 'peciaiassistants'), 21.389894397579603), (('_secreta', 'ry_of_state_robs_sterling_ba'), 21.389894397579603), (('a_', 'maveff'), 21.389894397579603), (('a_pia', 'ivene'), 21.389894397579603), (('aaad', 'htmly'), 21.389894397579603), (('aat', 'hacatiasierchassme'), 21.389894397579603), (('abba', 'eban'), 21.389894397579603), (('abdelrahim', 'oshi'), 21.389894397579603), (('abiola', 'farida'), 21.389894397579603), (('abis', 'syh'), 21.389894397579603), (('abot', 'theiweb'), 21.389894397579603), (('aca', 'demic'), 21.389894397579603), (('achtnn', 'nepdc'), 21.389894397579603), (('aciout', 'foreimi'), 21.389894397579603), (('acordado', 'continuar'), 21.389894397579603), (('acre', 'oasis')

In [7]:
#extract features for clusterizzation
from sklearn.feature_extraction.text import TfidfVectorizer
connected_texts = [" ".join(text) for text in texts]
counter = TfidfVectorizer(strip_accents='unicode', ngram_range=(2,2), stop_words='english', max_df=0.99, min_df=0.001)
A = counter.fit_transform(connected_texts).toarray()

In [8]:
#reduce dimenstions
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

print A.shape, len(texts)
svd = TruncatedSVD(50)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
A = lsa.fit_transform(A)

(7945, 13772) 7945


In [9]:
km = KMeans(n_clusters=50, init='k-means++', max_iter=500, n_init=100)
result = km.fit_predict(A)

In [15]:
#visualize results, print words with max sum of tf-idf in each cluster
counter_unigrams = TfidfVectorizer(strip_accents='unicode', ngram_range=(1,1), stop_words='english', max_df=0.98, min_df=0.01)
for i in xrange(50):
    print i, ":"
    cluster_texts = []
    for j, text in enumerate(connected_texts):
        if result[j] == i:
            cluster_texts.append(text)
    res = counter_unigrams.fit_transform(cluster_texts).toarray()
    res = np.sum(res, axis = 0)
    #invert vocabulary to index->word
    terms = {}
    for z in counter_unigrams.vocabulary_:
        terms[counter_unigrams.vocabulary_[z]] = z
    #10 top tf-idf words
    for k in xrange(10):
        j = np.unravel_index(res.argmax(), res.shape)[0]
        print terms[j]
        res[j] = 0
    print

0 :
haiti
united
clinton
said
states
israel
government
rights
secretary
president

1 :
list
huma
abedin
gov
cc
clintonemail
com
aug
tomorrow
pm

2 :
prince
sbu
dec
december
crown
abedinh
hussein
abdullah
monday
agencies

3 :
gov
message
original
cheryl
january
jan
haiti
pm
com
mills

4 :
gov
message
original
jun
cheryl
sullivan
pm
com
mills
clintonemail

5 :
subject
gov
jacob
pm
sullivan
message
original
cheryl
nov
com

6 :
http
com
www
israel
netanyahu
obama
said
new
subject
sent

7 :
cheryl
mills
pm
gov
millscd
message
june
original
august
kelly

8 :
today
calls
sat
ok
tomorrow
wed
sun
pm
talk
jun

9 :
subject
gov
pm
sunday
saturday
talk
thursday
october
september
called

10 :
libya
september
pm
cheryl
source
mills
libyan
sullivan
al
magariaf

11 :
gov
pm
sullivanjj
wednesday
tuesday
jilotylc
sunday
saturday
thursday
friday

12 :
huma
valmorou
abedin
com
clintonemail
schedule
pm
message
original
sunday

13 :
sullivan
jacob
gov
message
march
original
pm
speech
mar
com

14 :
subject
au

Кластеры уже похожи на интерпретируемые.

Пусть ассесоры просматривают письма из кластера и отвечают на вопрос: "письмам из кластера можно назначить общую тему?"
0 - нельзя
0.5 - нет сильной уверенности, что предложенная ассесорам тема их объеденяет
1 - ассесор уверен в теме, которую назначил кластеру

Результаты ассесора: 0.82